# Multi-Phase Production Forecasting

Demonstrates simultaneous forecasting of oil, gas, and water production.

## Key Features
- Multi-phase data structures
- Coupled vs independent forecasting
- Physical relationship maintenance (GOR, water cut)
- Consistency metrics

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from decline_analysis.multiphase import (
    MultiPhaseData,
    MultiPhaseForecaster,
    create_multiphase_data_from_dataframe
)

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. Load Bakken Data

In [ ]:
df = pd.read_csv('data/bakken_well_production.csv')
df['ReportDate'] = pd.to_datetime(df['ReportDate'])

data = create_multiphase_data_from_dataframe(
    df, oil_column='Oil', gas_column='Gas',
    water_column='Wtr', date_column='ReportDate'
)

print(f"Phases: {data.phases}")
print(f"Length: {data.length} months")
ratios = data.calculate_ratios()
print(f"Avg GOR: {ratios['gor'].mean():.1f} mcf/bbl")
print(f"Avg Water Cut: {ratios['water_cut'].mean():.1f}%")

## 2. Coupled Forecasting

In [ ]:
forecaster = MultiPhaseForecaster()

coupled = forecaster.forecast(
    data, horizon=24, model='arps',
    kind='hyperbolic', enforce_ratios=True
)

print("Coupled Forecast:")
for phase in ['oil', 'gas', 'water']:
    total = coupled[phase].iloc[len(data.oil):].sum()
    print(f"  {phase}: {total:,.0f}")

## 3. Independent Forecasting

In [ ]:
independent = forecaster.forecast(
    data, horizon=24, model='arps',
    kind='hyperbolic', enforce_ratios=False
)

print("Independent Forecast:")
for phase in ['oil', 'gas', 'water']:
    total = independent[phase].iloc[len(data.oil):].sum()
    print(f"  {phase}: {total:,.0f}")

## 4. Compare Accuracy

In [ ]:
coupled_metrics = forecaster.evaluate(data, coupled)
independent_metrics = forecaster.evaluate(data, independent)

print("Accuracy Comparison:")
for phase in ['oil', 'gas', 'water']:
    c_rmse = coupled_metrics[phase]['rmse']
    i_rmse = independent_metrics[phase]['rmse']
    improvement = (i_rmse - c_rmse) / i_rmse * 100
    print(f"\n{phase.upper()}:")
    print(f"  Coupled RMSE: {c_rmse:,.0f}")
    print(f"  Independent RMSE: {i_rmse:,.0f}")
    print(f"  Improvement: {improvement:.1f}%")

## 5. Check Consistency

In [ ]:
coupled_consistency = forecaster.calculate_consistency_metrics(coupled)
independent_consistency = forecaster.calculate_consistency_metrics(independent)

print("Consistency Metrics:")
print("\nCoupled:")
for k, v in coupled_consistency.items():
    print(f"  {k}: {v:.3f}" if isinstance(v, float) else f"  {k}: {v}")

print("\nIndependent:")
for k, v in independent_consistency.items():
    print(f"  {k}: {v:.3f}" if isinstance(v, float) else f"  {k}: {v}")

## 6. Visualize Results

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(14, 10))

for idx, (phase, color) in enumerate([('oil', 'green'), ('gas', 'red'), ('water', 'blue')]):
    ax = axes[idx]
    hist = getattr(data, phase)
    ax.plot(hist.index, hist.values, 'o-', color=color, label='Historical')
    
    c_future = coupled[phase].iloc[len(hist):]
    ax.plot(c_future.index, c_future.values, '--', color=color, label='Coupled')
    
    i_future = independent[phase].iloc[len(hist):]
    ax.plot(i_future.index, i_future.values, ':', color=color, label='Independent')
    
    ax.axvline(x=hist.index[-1], color='gray', linestyle=':')
    ax.set_ylabel(phase.capitalize())
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()